In [ ]:
# This code is to be ran on Google Colab, and it clones a specific branch of a GitHub repository.
!git clone -b main https://github.com/IgnacioOQ/biased_recommending
%cd biased_recommending
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

dumping_path = '/content/drive/My Drive/Colab Projects/Data Driven ABMs/Data Sets/ignacio_playground/'
print("Current Directory:", dumping_path)

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add src to path if not already (for notebook execution)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.simulation import GameSession

# --- Game State ---
# Note: In Colab, the output_dir should be set to the Drive path if desired, 
# but for local testing 'data' is fine. Users on Colab can change this arg.
game = GameSession(output_dir="data")
current_recs = game.start_game()
current_step_info = None

# History for plotting
history_scores = []
history_episodes = []
cumulative_score = 0

# --- UI Elements ---
output_plot = widgets.Output()

header_html = widgets.HTML(value="<h1>Recommender Game</h1>")
instructions = widgets.HTML(value="""
<p><b>Goal:</b> Maximize your score!</p>
<p>1. Observe the recommendations from Agent 1 and Agent 2.</p>
<p>2. Choose which agent to follow.</p>
<p>3. See the outcome (Heads/Tails) and your reward.</p>
<hr>
""")

agent1_btn = widgets.Button(description="Follow Agent 1", button_style='info')
agent2_btn = widgets.Button(description="Follow Agent 2", button_style='info')
next_btn = widgets.Button(description="Next Step", disabled=True)

score_label = widgets.Label(value="Score: 0 | Episode: 0 | Step: 0")
feedback_label = widgets.HTML(value="")
metrics_label = widgets.HTML(value="")

# --- Logic ---

def update_plot():
    with output_plot:
        clear_output(wait=True)
        if not history_scores:
            return
            
        plt.figure(figsize=(10, 4))
        plt.plot(history_scores, label="Cumulative Score")
        plt.xlabel("Step")
        plt.ylabel("Score")
        plt.title("Performance Over Time")
        plt.legend()
        plt.grid(True)
        plt.show()

def update_ui(recommendations):
    rec_text = ["Recommend" if r == 1 else "Not Recommend" for r in recommendations]
    
    agent1_btn.description = f"Agent 1: {rec_text[0]}"
    agent2_btn.description = f"Agent 2: {rec_text[1]}"
    
    agent1_btn.disabled = False
    agent2_btn.disabled = False
    next_btn.disabled = True
    
    # Visual cues
    agent1_btn.icon = 'thumbs-up' if recommendations[0] == 1 else 'thumbs-down'
    agent2_btn.icon = 'thumbs-up' if recommendations[1] == 1 else 'thumbs-down'

def on_choice(b):
    global cumulative_score, current_step_info
    
    choice = 0 if b == agent1_btn else 1
    
    # Process Step
    current_step_info = game.process_step(choice)
    
    # Update Score
    reward = current_step_info['human_reward']
    cumulative_score += reward
    
    # Update History
    history_scores.append(cumulative_score)
    update_plot()
    
    # Show Feedback
    outcome = current_step_info['outcome']
    rec_followed = "Recommend" if game.current_recommendations[choice] == 1 else "Not Recommend"
    
    res_color = "green" if reward > 0 else "red"
    
    feedback_html = f"""
    <div style="border: 2px solid {res_color}; padding: 10px; border-radius: 5px;">
        <h3>Outcome: {outcome}</h3>
        <p>You followed Agent {choice + 1} ({rec_followed}).</p>
        <p><b>Reward: {reward}</b></p>
    </div>
    """
    feedback_label.value = feedback_html
    
    # Update Status Bar
    score_label.value = f"Score: {cumulative_score} | Episode: {current_step_info['episode_count']} | Step: {game.env.steps}"
    
    # Disable choice buttons, enable Next
    agent1_btn.disabled = True
    agent2_btn.disabled = True
    next_btn.disabled = False
    
    # Check for metrics (End of Episode)
    if current_step_info['metrics']:
        m = current_step_info['metrics']
        # Display Agent 0 metrics as sample
        a0 = m['agent_0']
        a1 = m['agent_1']
        metrics_html = f"""
        <div style="background-color: #f0f0f0; padding: 10px; margin-top: 10px;">
            <h4>Episode {current_step_info['episode_count']-1} Analysis</h4>
            <p><b>Agent 1 Disagreement with Unbiased:</b> {a0['disagreement_rate']:.2%}</p>
            <p><b>Agent 2 Disagreement with Unbiased:</b> {a1['disagreement_rate']:.2%}</p>
        </div>
        """
        metrics_label.value = metrics_html

def on_next(b):
    global current_recs
    
    feedback_label.value = ""
    # info contains recommendations for the *next* step already
    if current_step_info:
        current_recs = current_step_info['recommendations']
        update_ui(current_recs)
        
    if current_step_info and current_step_info['new_episode']:
        feedback_label.value = "<b>New Episode Started!</b>"

agent1_btn.on_click(on_choice)
agent2_btn.on_click(on_choice)
next_btn.on_click(on_next)

# --- Layout ---
ui = widgets.VBox([
    header_html,
    instructions,
    score_label,
    output_plot,
    widgets.HBox([agent1_btn, agent2_btn]),
    feedback_label,
    metrics_label,
    next_btn
])

# Initialize
update_ui(current_recs)

display(ui)
